In [ ]:
#importer les librairies
import os
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np
import seaborn as sns


In [ ]:
#redimensionner les images test
def resize_images(dossier, output, size):
    # Parcourir tous les sous-dossiers et fichiers du dossier principal
    for subdir, _, files in os.walk(dossier):
        for file in files:
            file_path = os.path.join(subdir, file)
            # Lire l'image
            img = cv2.imread(file_path)
            # Redimensionner l'image
            img_resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)

            # Construire le chemin de output
            relative_path = os.path.relpath(subdir, dossier) #copier l'arborescence
            output_subdir = os.path.join(output, relative_path)
            os.makedirs(output_subdir, exist_ok=True)
            output_path = os.path.join(output_subdir, file)
            cv2.imwrite(output_path, img_resized)

In [ ]:
data = '/content/drive/MyDrive/python dm/data/train/test' #chemin vers votre dossier test
output = '/content/drive/MyDrive/python dm/data/train/map_red' #créer un nouveau dossier pour enregistrer les images redimensionnées
resize_images(data,output,size=(64, 64)) #vous pouvez essayer d'autres tailles

In [ ]:
grec = YOLO('/content/drive/MyDrive/python dm/v8s_20epoch/classify/train/weights/best.pt')
grec.names

In [ ]:
# Mapping des noms de sous-dossiers à leurs indices, c'est le numéro de classe des lettres obtenu à partir de grec.names
class_alphabet = {'Α': 0, 'Ε': 1, 'I': 2, 'N': 3, 'O': 4, 'Β': 5, 'Γ': 6, 'Δ': 7, 'Ζ': 8, 'Η': 9,
                     'Θ': 10, 'Κ': 11, 'Λ': 12, 'Μ': 13, 'Ξ': 14, 'Π': 15, 'Ρ': 16, 'Τ': 17, 'Υ': 18,
                     'Φ': 19, 'Χ': 20, 'Ψ': 21, 'Ω': 22, 'Ϲ': 23}

In [ ]:
def generate_occlusion(image, model, class_id, occlusion_size, step_size):
    image_height, image_width, _ = image.shape
    heatmap = np.zeros((image_height, image_width))


    predict_model = YOLO(model)
    results = predict_model.predict(image)

    if class_id in results[0].probs.top5:
        rank = results[0].probs.top5.index(class_id)
        initial_confidence = results[0].probs.top5conf[rank]
    else:
        initial_confidence = 0

    for y in range(0, image_width - occlusion_size[1] + 1, step_size):
        for x in range(0, image_height - occlusion_size[0] + 1, step_size):

            # Copier l'image pour ne pas modifier l'originale
            occluded_image = image.copy()

            # Appliquer l'occlusion
            occluded_image[x:x + occlusion_size[0], y:y + occlusion_size[1]] = 0

            # Prédire avec l'image occluse
            results = predict_model.predict(occluded_image)

            if class_id in results[0].probs.top5:
                rank = results[0].probs.top5.index(class_id)
                class_confidence = results[0].probs.top5conf[rank]
            else:
                class_confidence = 0

            heatmap[x:x + occlusion_size[0], y:y + occlusion_size[1]] = initial_confidence - class_confidence

    # Normalisation du heatmap
    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))

    return heatmap

In [ ]:
def generate_heatmaps(folder_path, model_path, class_alphabet, occlusion_size, step_size):
    heatmaps = []
    # Parcourir tous les sous-dossiers dans le dossier principal
    for subfolder_name in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder_name)

            # Parcourir tous les fichiers du sous-dossier
        for filename in os.listdir(subfolder_path):
                # Chemin complet de l'image
            image_path = os.path.join(subfolder_path, filename)

                # Lire l'image
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)

                # Récupérer l'indice du sous-dossier à partir du mapping
            class_id = class_alphabet.get(subfolder_name)

                # Générer le heatmap
            heatmap = generate_occlusion(image, model_path, class_id, occlusion_size, step_size)
            heatmaps.append(heatmap)

    return heatmaps


In [ ]:
folder_path = "/content/drive/MyDrive/python dm/data/train/map_red"
model_path = '/content/drive/MyDrive/python dm/v8s_20epoch/classify/train/weights/best.pt'
occlusion_size = (8,8) #ou 5,5 ou autres
step_size = 8 #ou 5 ou autres
#5*5 est plus précis que 8*8, mais plus lourd à charger pour toutes les images
heatmaps = generate_heatmaps(folder_path, model_path, class_alphabet, occlusion_size, step_size)


In [ ]:
def is_valid_heatmap(heatmap):
    # Vérifier qu'il n'y a pas de NaN
    if not np.all(np.isfinite(heatmap)):
        return False
    # Vérifier que la heatmap a les même dimensions
    if heatmap.shape != (64, 64):
        return False
    return True

def superposer(heatmaps):
    # Filtrer les heatmaps non valides
    valid_heatmaps = [heatmap for heatmap in heatmaps if is_valid_heatmap(heatmap)]

    # Initialisation d'une heatmap vide pour la somme
    combined_heatmap = np.zeros_like(valid_heatmaps[0], dtype=float)

    # Normaliser et ajouter chaque heatmap valide à la somme
    for heatmap in valid_heatmaps:
        normalized_heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))
        combined_heatmap += normalized_heatmap

    # Calculer la moyenne
    combined_heatmap /= len(valid_heatmaps)

    return combined_heatmap

In [ ]:
superposer_heatmap = superposer(heatmaps)

In [ ]:
plt.imshow(superposer_heatmap, cmap='YlGn')
plt.colorbar()
plt.show()
#plt.savefig()

In [ ]:
#superposer la carte sur une image
def overlay_heatmap(image, heatmap, alpha=0.6):
    heatmap_normalized = (heatmap - np.min(heatmap) / np.max(heatmap) - np.min(heatmap))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_normalized), cv2.COLORMAP_JET)

    overlay = cv2.addWeighted(image, alpha, heatmap_colored, 1 - alpha, 0)
    return overlay

In [ ]:
overlay_image = overlay_heatmap(image, superposer_heatmap)
plt.imshow(cv2.cvtColor(overlay_image, cv2.COLOR_BGR2RGB))
plt.show()